In [3]:
from pyparsing import Word, alphas, nums, Literal, Or, ParseException, ZeroOrMore, Group, delimitedList, OneOrMore

identifier = Word(alphas)
value = Word(nums)
variable = identifier ^ value

clause = identifier.setResultsName("predicate") + "(" + delimitedList(variable, delim=",").setResultsName("variables") + ")"
rule = clause.setResultsName("head") + ":-" + delimitedList(Group(clause), delim=",").setResultsName("body")
statement = Group((clause ^ rule) + Or([".", "?"]))
program = OneOrMore(statement)

In [6]:
from collections import defaultdict
from copy import deepcopy

def is_variable(v):
    return v[0].isalpha() and v[0].isupper()

def find_matching_indices(l1, l2):
    r = list()
    for el in l2:
        if el in l1:
            r.append(l1.index(el))
        else:
            r.append(None)
    return r

def imply(memory, s):
    '''
    For each clause in the input statement, imply uses a database of known facts
    (the memory) to generate all possible substitutions that satisfy that clause.
    If a valid substitution is found, a new statement is generated with the
    substitution propagated to all other clauses and the satisfied clause removed.
    If a statement runs out of body clauses then it must be that all its clauses
    have been satisfied and all substitutions made, so the head can be added to
    fact database.
    '''
    new_statements = list()
    if "head" in s:
        locs = variable_locations(s)
        if "assignments" not in s:
            s["assignments"] = defaultdict(lambda:None)
        if len(s["body"]) == 0:
            memory[s["head"]["predicate"]].append(s["head"]["variables"])
        for cidx in range(len(s["body"])):
            c = s["body"][cidx]
            if any(is_variable(var) for var in c["variables"]):
                for values in memory[c["predicate"]]:
                    new_s = deepcopy(s)
                    try:
                        substitutions = generate_substitutions(c, values)
                        new_s = make_substitutions(new_s, locs, substitutions)
                        new_s["body"] = new_s["body"][:cidx] + new_s["body"][cidx+1:]
                        new_statements.append(new_s)
                    except SubstitutionConflict as e:
                        # if there is a substitution conflict, move on without adding
                        # any new statements
                        pass
            else:
                # if the clause matches some fact in memory, it has already been
                # fulfilled so we can just remove it
                if clause_is_valid(memory, c):
                    new_s = deepcopy(s)
                    new_s["body"] = new_s["body"][:cidx] + new_s["body"][cidx+1:]
                    new_statements.append(new_s)
    else:
        memory[s["predicate"]].append(s["variables"])
    return new_statements

def clause_is_valid(memory, clause):
    '''Searches for valid match of a fully-substituted clause in the fact database.'''
    valid = False
    for values in memory[clause["predicate"]]:
        if all(values[i] == clause["variables"][i] for i in range(len(values))):
            valid = True
            break
    return valid

def clause_to_string(clause):
    return clause["predicate"] + "(" + ", ".join(clause["variables"]) + ")"

def rule_to_string(rule):
    return clause_to_string(rule["head"]) + " :- " + ", ".join(
        clause_to_string(bc) for bc in rule["body"])

class SubstitutionConflict(Exception):
    pass

def generate_substitutions(clause, new_values):
    '''
    Generates a candidate substitution for each variable in the clause and
    throws an error if the candidate violates a pre-existing value.
    '''
    candidates = dict(zip(clause["variables"], new_values))
    # trying to make substitutions
    for vidx in range(len(clause["variables"])):
        var = clause["variables"][vidx]
        if not is_variable(var):
            # found prior value, check that new value is consistent with it
            if var != new_values[vidx]:
                raise SubstitutionConflict("Conflicting values: " +
                                           var + value)
    return candidates

def make_substitutions(rule, locs, substitutions):
    '''Modifies the actual statements so as to effect the substitution.'''
    for var, value in substitutions.items():
        if rule["assignments"][var] is None:
            for vidx in locs[var]["head"]:
                rule["head"]["variables"][vidx] = value
            for cidx in range(len(locs[var]["body"])):
                c = locs[var]["body"][cidx]
                for vidx in c:
                    rule["body"][cidx]["variables"][vidx] = value
        else:
            # found prior value, check that new value is consistent with it
            if rule["assignments"][var] != value:
                raise SubstitutionConflict("Conflicting values: " +
                                           rule["assignments"][var] + value)
    return rule

def variable_locations(rule):
    '''
    Returns an array of variable locations, where locs[varname]['head'] is
    a list of all the locations where the variable is present in the head of
    the statement, and locs[varname]['body'] is a list of lists of locations
    for each clause in the body.
    '''
    num_clauses = len(rule["body"])
    locs = defaultdict(lambda:{"head":list(), "body":[[] for i in range(num_clauses)]})
    for cidx in range(len(rule["body"])):
        clocs = defaultdict(list)
        for vidx in range(len(rule["body"][cidx]["variables"])):
            varname = rule["body"][cidx]["variables"][vidx]
            clocs[varname].append(vidx)
        for varname, l in clocs.items():
            locs[varname]["body"][cidx] = l
    for vidx in range(len(rule["head"]["variables"])):
        varname = rule["head"]["variables"][vidx]
        locs[varname]["head"].append(vidx)
    return locs
            
def evaluate(statements):
    statements = [s.asDict() for s in statements]
    memory = defaultdict(list)
    while len(statements) > 0:
        new_statements = imply(memory, statements[0])
        statements = statements[1:] + new_statements
    return memory

def run(programString):
    return evaluate(program.parseString(programString))

In [13]:
run("""
insert(list, 1).
insert(stack, 1).
insert(heap, logn).

build(X, T, 1) :- insert(X, T).
build(X, T, 2) :- insert(X, T), build(X, T, 1).
""")['build']

[['list', '1', '1'],
 ['stack', '1', '1'],
 ['heap', 'logn', '1'],
 ['list', '1', '2'],
 ['stack', '1', '2'],
 ['heap', 'logn', '2']]

In [ ]:
run("""
first(s, m, 0).
last() :- last(s, p, i)

""")